# RMSProp
:label:`sec_rmsprop`

One of the key issues in :numref:`sec_adagrad` is that the learning rate decreases at a predefined schedule of effectively $\mathcal{O}(t^{-\frac{1}{2}})$. While this is generally appropriate for convex problems, it might not be ideal for nonconvex ones, such as those encountered in deep learning. Yet, the coordinate-wise adaptivity of Adagrad is highly desirable as a preconditioner.

:cite:`Tieleman.Hinton.2012` proposed the RMSProp algorithm as a simple fix to decouple rate scheduling from coordinate-adaptive learning rates. The issue is that Adagrad accumulates the squares of the gradient $\mathbf{g}_t$ into a state vector $\mathbf{s}_t = \mathbf{s}_{t-1} + \mathbf{g}_t^2$. As a result $\mathbf{s}_t$ keeps on growing without bound due to the lack of normalization, essentially linarly as the algorithm converges.

One way of fixing this problem would be to use $\mathbf{s}_t / t$. For reasonable distributions of $\mathbf{g}_t$ this will converge. Unfortunately it might take a very long time until the limit behavior starts to matter since the procedure remembers the full trajectory of values. An alternative is to use a leaky average in the same way we used in the momentum method, i.e., $\mathbf{s}_t \leftarrow \gamma \mathbf{s}_{t-1} + (1-\gamma) \mathbf{g}_t^2$ for some parameter $\gamma > 0$. Keeping all other parts unchanged yields RMSProp.

## The Algorithm

Let us write out the equations in detail.

$$\begin{aligned}
    \mathbf{s}_t & \leftarrow \gamma \mathbf{s}_{t-1} + (1 - \gamma) \mathbf{g}_t^2, \\
    \mathbf{x}_t & \leftarrow \mathbf{x}_{t-1} - \frac{\eta}{\sqrt{\mathbf{s}_t + \epsilon}} \odot \mathbf{g}_t.
\end{aligned}$$

The constant $\epsilon > 0$ is typically set to $10^{-6}$ to ensure that we do not suffer from division by zero or overly large step sizes. Given this expansion we are now free to control the learning rate $\eta$ independently of the scaling that is applied on a per-coordinate basis. In terms of leaky averages we can apply the same reasoning as previously applied in the case of the momentum method. Expanding the definition of $\mathbf{s}_t$ yields

$$
\begin{aligned}
\mathbf{s}_t & = (1 - \gamma) \mathbf{g}_t^2 + \gamma \mathbf{s}_{t-1} \\
& = (1 - \gamma) \left(\mathbf{g}_t^2 + \gamma \mathbf{g}_{t-1}^2 + \gamma^2 \mathbf{g}_{t-2} + \ldots, \right).
\end{aligned}
$$

As before in :numref:`sec_momentum` we use $1 + \gamma + \gamma^2 + \ldots, = \frac{1}{1-\gamma}$. Hence the sum of weights is normalized to $1$ with a half-life time of an observation of $\gamma^{-1}$. Let us visualize the weights for the past 40 timesteps for various choices of $\gamma$.


In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.GradDescUtils.plotGammas
import jp.live.ugai.d2j.util.GradDescUtils.train2d
import jp.live.ugai.d2j.util.GradDescUtils.showTrace2d
import jp.live.ugai.d2j.util.TrainingChapter11.getDataCh11
import jp.live.ugai.d2j.util.TrainingChapter11.trainCh11
import jp.live.ugai.d2j.util.TrainingChapter11.trainConciseCh11
import jp.live.ugai.d2j.util.LossTime
import org.jetbrains.letsPlot.intern.Plot

In [2]:
val manager = NDManager.newBaseManager();

val gammas = listOf(0.95f, 0.9f, 0.8f, 0.7f)

val timesND = manager.arange(40f);
val times = timesND.toFloatArray().toList()
plotGammas(times, gammas, 600, 400)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0]


## Implementation from Scratch

As before we use the quadratic function $f(\mathbf{x})=0.1x_1^2+2x_2^2$ to observe the trajectory of RMSProp. Recall that in :numref:`sec_adagrad`, when we used Adagrad with a learning rate of 0.4, the variables moved only very slowly in the later stages of the algorithm since the learning rate decreased too quickly. Since $\eta$ is controlled separately this does not happen with RMSProp.


In [7]:
var eta = 0.4f;
var gamma = 0.9f;

val rmsProp2d = { state: List<Float> -> 
    var x1 = state[0]
    var x2 = state[1]
    var s1 = state[2]
    var s2 = state[3]
    var g1 = 0.2f * x1
    var g2 = 4.0f * x2
    var eps = 1e-6.toFloat()
    s1 = gamma * s1 + (1 - gamma) * g1 * g1;
    s2 = gamma * s2 + (1 - gamma) * g2 * g2;
    x1 -= eta / Math.sqrt(s1.toDouble() + eps).toFloat() * g1
    x2 -= eta / Math.sqrt(s2.toDouble() + eps).toFloat() * g2;
    listOf(x1, x2, s1, s2)
}

val f2d = fun(x1:Float, x2: Float) : Float {
    return 0.1f * x1 * x1 + 2 * x2 * x2;
}

showTrace2d(f2d, train2d(rmsProp2d, 20));

Tablesaw not supporting for contour and meshgrids, will update soon


![RmsProp Gradient Descent 2D.](https://d2l-java-resources.s3.amazonaws.com/img/chapter_optim-rmsprop-gd2d.svg)

Next, we implement RMSProp to be used in a deep network. This is equally straightforward.


In [8]:
fun initRmsPropStates(featureDimension: Int) : NDList {
    val manager = NDManager.newBaseManager();
    val sW = manager.zeros(Shape(featureDimension.toLong(), 1));
    val sB = manager.zeros(Shape(1));
    return NDList(sW, sB);
}

object Optimization {
    fun rmsProp(params:NDList , states:NDList , hyperparams:Map<String, Float> ) {
        val gamma = hyperparams.get("gamma")!!
        val eps = 1e-6.toFloat()
        for (i in 0 until params.size) {
            val param = params.get(i);
            val state = states.get(i);
            // Update parameter and state
            // state = gamma * state + (1 - gamma) * param.gradient^(1/2)
            state.muli(gamma).addi(param.getGradient().square().mul(1 - gamma));
            // param -= lr * param.gradient / sqrt(s + eps)
            param.subi(param.getGradient().mul(hyperparams.get("lr")).div(state.add(eps).sqrt()));
        }
    }
}

We set the initial learning rate to 0.01 and the weighting term $\gamma$ to 0.9. That is, $\mathbf{s}$ aggregates on average over the past $1/(1-\gamma) = 10$ observations of the square gradient.


In [10]:
val airfoil = getDataCh11(10, 1500);

fun trainRmsProp(lr: Float, gamma: Float, numEpochs: Int) : LossTime {
    var featureDimension = airfoil.getColumnNames().size
    var hyperparams = mutableMapOf<String, Float>();
    hyperparams.put("lr", lr);
    hyperparams.put("gamma", gamma)
    return trainCh11(Optimization::rmsProp, 
                                       initRmsPropStates(featureDimension), 
                                       hyperparams, airfoil, 
                                       featureDimension, numEpochs)
}

trainRmsProp(0.01f, 0.9f, 2)

loss: 0.245, 0.159 sec/epoch


jp.live.ugai.d2j.util.LossTime@1ec7d8b3

## Concise Implementation

Since RMSProp is a rather popular algorithm it is also available in `Optimizer`. We create an instance of `RmsProp` and set its learning rate and optional `gamma1` parameter. 

In [11]:
val lrt = Tracker.fixed(0.01f);
val rmsProp = Optimizer.rmsprop().optLearningRateTracker(lrt).optRho(0.9f).build();

trainConciseCh11(rmsProp, airfoil, 2);

Training:    100% |████████████████████████████████████████| Accuracy: 0.67, L2Loss: 0.28�██████████████████████████████████   | Accuracy: 0.67, L2Loss: 0.29, L2Loss: 0.28
loss: 0.246, 0.267 sec/epoch


jp.live.ugai.d2j.util.LossTime@5bb3131b

## Summary

* RMSProp is very similar to Adagrad insofar as both use the square of the gradient to scale coefficients.
* RMSProp shares with momentum the leaky averaging. However, RMSProp uses the technique to adjust the coefficient-wise preconditioner.
* The learning rate needs to be scheduled by the experimenter in practice.
* The coefficient $\gamma$ determines how long the history is when adjusting the per-coordinate scale.

## Exercises

1. What happens experimentally if we set $\gamma = 1$? Why?
1. Rotate the optimization problem to minimize $f(\mathbf{x}) = 0.1 (x_1 + x_2)^2 + 2 (x_1 - x_2)^2$. What happens to the convergence?
1. Try out what happens to RMSProp on a real machine learning problem, such as training on Fashion-MNIST. Experiment with different choices for adjusting the learning rate.
1. Would you want to adjust $\gamma$ as optimization progresses? How sensitive is RMSProp to this?
